# Word Vectorization

Ariel Silbert

Sources LSA:
- https://machinelearninggeek.com/latent-semantic-indexing-using-scikit-learn/
- https://machinelearninggeek.com/discovering-hidden-themes-of-documents/ (some sources doubt if this is a useful measure)
- https://towardsdatascience.com/latent-semantic-analysis-deduce-the-hidden-topic-from-the-document-f360e8c0614b
- course material from GWU's Information Systems class

Sources word vectors:
- https://www.analyticsvidhya.com/blog/2021/08/a-friendly-guide-to-nlp-bag-of-words-with-python-example/
- https://towardsdatascience.com/understanding-nlp-word-embeddings-text-vectorization-1a23744f7223

Run with data collected through April 23

In [1]:
import pandas as pd
import json
import os
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
import operator
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
!pip install --upgrade gensim
from gensim import corpora
from gensim.models import LsiModel
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

You should consider upgrading via the '/Users/arielsilbert/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
'''video_stats = pd.DataFrame()

directory = 'TikTokImpact-main/TikTok_scrape/output'
for datefolder in os.listdir(directory): #go into folders by date
    f = os.path.join(directory, datefolder)
    if os.path.isdir(f): #checking if it is a directory
        for subfolder in os.listdir(f): #go into subfolders
            g = os.path.join(f, subfolder)
            if os.path.isdir(g):
                for json_file in os.listdir(g): #go into video and user stats folders
                    h = os.path.join(g, json_file)
                    if 'OrigSearch' in h:
                        data = json.load(open(h))
                        data['date'] = int(h.split('/')[3].split('_')[0]) #select date from folder name
                        df = pd.json_normalize(data)
                        video_stats = video_stats.append(df) #not an efficient way of loading the data to the df'''

video_stats = pd.read_csv(r'video_stats.csv')

In [3]:
video_stats.head()

id                                               desc  \
0  7069787462242962694                                   #ukraine #russia   
1  7082556143888878890       #ukraine #ukrainevsrussia #ukrainewar #bucha   
2  7068388771854634245  I wish this is a bad dream and we will wake up...   
3  7082576425911323946                         its exciting every timeeee   
4  7071700963953741062        🇺🇦🤝 🇷🇺 #ukraine #supportukraine #fyp #nowar   

   createTime                                         challenges  \
0  1646063166  [{'id': '451042', 'title': 'ukraine', 'desc': ...   
1  1649036106  [{'id': '451042', 'title': 'ukraine', 'desc': ...   
2  1645737509  [{'id': '861413', 'title': 'stopwar', 'desc': ...   
3  1649040828                                                NaN   
4  1646508688  [{'id': '451042', 'title': 'ukraine', 'desc': ...   

   originalItem  officalItem  \
0         False        False   
1         False        False   
2         False        False   
3         False        False   
4         False        False   

                                           textExtra  secret  forFriend  \
0  [{'awemeId': '', 'start': 0, 'end': 8, 'hashta...   False      False   
1  [{'awemeId': '', 'start': 0, 'end': 8, 'hashta...   False      False   
2  [{'awemeId': '', 'start': 56, 'end': 64, 'hash...   False      False   
3                                                NaN   False      False   
4  [{'awemeId': '', 'start': 12, 'end': 20, 'hash...   False      False   

   digged  ...  authorStats.followingCount  authorStats.followerCount  \
0   False  ...                           0                     165100   
1   False  ...                           4                        204   
2   False  ...                          63                     124400   
3   False  ...                        6098                       1652   
4   False  ...                          32                   10800000   

   authorStats.heartCount  authorStats.videoCount  authorStats.diggCount  \
0                 4700000                     910                    548   
1                    1121                     145                    135   
2                 3100000                      69                     61   
3                   18800                    1662                  84700   
4               226600000                     701                   3361   

   authorStats.heart  maskType  effectStickers author.roomId  warnInfo  
0            4700000       NaN             NaN           NaN       NaN  
1               1121       3.0             NaN           NaN       NaN  
2            3100000       NaN             NaN           NaN       NaN  
3              18800       NaN             NaN           NaN       NaN  
4          226600000       NaN             NaN           NaN       NaN  

[5 rows x 80 columns]

In [4]:
video_stats.columns

Index(['id', 'desc', 'createTime', 'challenges', 'originalItem', 'officalItem',
       'textExtra', 'secret', 'forFriend', 'digged', 'itemCommentStatus',
       'showNotPass', 'vl1', 'itemMute', 'privateItem', 'duetEnabled',
       'stitchEnabled', 'shareEnabled', 'stickersOnItem', 'isAd', 'date',
       'video.id', 'video.height', 'video.width', 'video.duration',
       'video.ratio', 'video.cover', 'video.originCover', 'video.dynamicCover',
       'video.playAddr', 'video.downloadAddr', 'video.shareCover',
       'video.reflowCover', 'video.bitrate', 'video.encodedType',
       'video.format', 'video.videoQuality', 'video.encodeUserTag',
       'author.id', 'author.uniqueId', 'author.nickname', 'author.avatarThumb',
       'author.avatarMedium', 'author.avatarLarger', 'author.signature',
       'author.verified', 'author.secUid', 'author.secret', 'author.ftc',
       'author.relation', 'author.openFavorite', 'author.commentSetting',
       'author.duetSetting', 'author.stitchSetting'

In [5]:
video_stats.desc, video_stats.id

(0                                         #ukraine #russia
 1             #ukraine #ukrainevsrussia #ukrainewar #bucha
 2        I wish this is a bad dream and we will wake up...
 3                               its exciting every timeeee
 4              🇺🇦🤝 🇷🇺 #ukraine #supportukraine #fyp #nowar
                                ...                        
 19865    #foryourpage #vladimirputin #shocking #murtyme...
 19866    Le présentent ukrainien dénonce l'attaque cont...
 19867         Idk how else to say it #ukraine #russia #ww3
 19868                  Dayum #fyp #xyzbca #ukraine #russia
 19869    what’s this goofy ahh sound bruh 💀💀 #goofy #go...
 Name: desc, Length: 19870, dtype: object,
 0        7069787462242962694
 1        7082556143888878890
 2        7068388771854634245
 3        7082576425911323946
 4        7071700963953741062
                 ...         
 19865    7070296231565692161
 19866    7072706328795368709
 19867    7068763810110164270
 19868    70684206501590172

In [77]:
# Initialize regex tokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Vectorize document using TF-IDF
tfidf = TfidfVectorizer(lowercase=True,
                        stop_words='english',
                        ngram_range = (1,1),
                        tokenizer = tokenizer.tokenize)

# Fit and Transform the documents
train_data = tfidf.fit_transform(list(video_stats.desc.fillna(' '))) #without some text, will not analyze

In [78]:
# Define the number of topics or components
num_components=4
cols = []
for i in range(num_components):
    cols.append(f'topic_{i+1}')

# Create SVD object
lsa = TruncatedSVD(n_components=num_components, n_iter=100, random_state=42)

# Fit SVD model on data
svd_model = lsa.fit_transform(train_data)

# Get Singular values and Components 
Sigma = lsa.singular_values_ 
V_transpose = lsa.components_.T

In [83]:
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(svd_model, columns = cols)
for ele in video_stats.columns:
    topic_encoded_df[ele] = video_stats[ele]
topic_encoded_df.to_csv('video_stats_withLSA.csv', index = False)

In [85]:
topic_encoded_df

topic_1             topic_2             topic_3  \
0      0.7516497683180823  0.3127699285566003  0.2656970026962326   
1      0.0943428852334947  0.0784585542905636 -0.0323089444599723   
2      0.0609865198433351  0.0685462092780650 -0.0467847738714326   
3     -0.0000000000000000 -0.0000000000000000 -0.0000000000000000   
4      0.1981006286326522 -0.0201352131724603 -0.1276679799274636   
...                   ...                 ...                 ...   
19865  0.0709074984029047  0.0338242691323233 -0.0233271965942668   
19866  0.0653010377287930  0.0285217959593649  0.0149026910905254   
19867  0.2301939576273500  0.0103514555366069  0.1090209532353310   
19868  0.2454874389521169 -0.0531753358092679  0.0047066912819900   
19869  0.0407071235121972  0.0402887606615865  0.0036254578762862   

                  topic_4                   id  \
0     -0.1274365136081042  7069787462242962694   
1      0.0189513522102542  7082556143888878890   
2      0.0031934997920658  7068388771854634245   
3      0.0000000000000000  7082576425911323946   
4     -0.1352723976617556  7071700963953741062   
...                   ...                  ...   
19865  0.0010002641134518  7070296231565692161   
19866 -0.0083245208285983  7072706328795368709   
19867  0.0287034327454799  7068763810110164270   
19868 -0.1530328063641520  7068420650159017262   
19869  0.0019106190347089  7068904190533274923   

                                                    desc  createTime  \
0                                       #ukraine #russia  1646063166   
1           #ukraine #ukrainevsrussia #ukrainewar #bucha  1649036106   
2      I wish this is a bad dream and we will wake up...  1645737509   
3                             its exciting every timeeee  1649040828   
4            🇺🇦🤝 🇷🇺 #ukraine #supportukraine #fyp #nowar  1646508688   
...                                                  ...         ...   
19865  #foryourpage #vladimirputin #shocking #murtyme...  1646181627   
19866  Le présentent ukrainien dénonce l'attaque cont...  1646742768   
19867       Idk how else to say it #ukraine #russia #ww3  1645824828   
19868                Dayum #fyp #xyzbca #ukraine #russia  1645744930   
19869  what’s this goofy ahh sound bruh 💀💀 #goofy #go...  1645857514   

                                              challenges  originalItem  \
0      [{'id': '451042', 'title': 'ukraine', 'desc': ...         False   
1      [{'id': '451042', 'title': 'ukraine', 'desc': ...         False   
2      [{'id': '861413', 'title': 'stopwar', 'desc': ...         False   
3                                                    NaN         False   
4      [{'id': '451042', 'title': 'ukraine', 'desc': ...         False   
...                                                  ...           ...   
19865  [{'id': '1598498371111942', 'title': 'foryourp...         False   
19866  [{'id': '1494657', 'title': 'wwiii', 'desc': '...         False   
19867  [{'id': '451042', 'title': 'ukraine', 'desc': ...         False   
19868  [{'id': '229207', 'title': 'fyp', 'desc': '', ...         False   
19869  [{'id': '6837', 'title': 'goofy', 'desc': '', ...         False   

       officalItem  ... authorStats.followingCount  authorStats.followerCount  \
0            False  ...                          0                     165100   
1            False  ...                          4                        204   
2            False  ...                         63                     124400   
3            False  ...                       6098                       1652   
4            False  ...                         32                   10800000   
...            ...  ...                        ...                        ...   
19865        False  ...                          0                     187600   
19866        False  ...                         10                         71   
19867        False  ...                        895                       1301   
19868        False  ... 

In [84]:
#apparently the video_stats file is too big to upload to github in one go
for i in range(6):
    topic_encoded_df[i*3900:(i+1)*3900].to_csv(f'video_stats_withLSA_pt{i}.csv', index = False)

In [68]:
# Print the topics with their terms
terms = tfidf.get_feature_names()

for index, component in enumerate(lsa.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:5]
    top_terms_list=list(dict(top_terms_key).keys())
    print("Topic "+str(index+1)+": ",top_terms_list)
    
#after four topics, start to get words like "foryoupage" indicating loss of value for those topics

Topic 1:  ['ukraine', 'russia', 'fyp', 'war', 'ww3']
Topic 2:  ['ukraine', 'geopolitics', 's', 'invasion', 'ukrainian']
Topic 3:  ['russia', 'putin', 'news', 's', 'geopolitics']
Topic 4:  ['war', 'putin', 'saveukraine', 'nato', 'kyiv']


## Word Vectors (BOW)

In [16]:
#turn string descriptions into lists of words
desc_dict = {}

for key, val in zip(video_stats.id, video_stats.desc):

   #create list for the key
    desc_dict[key] = []
    # typecaste each val to string 
    val = str(val) 

   # split the value 
    tokens = val.split(" ") #will treat numerous spaces as words

    # Converts each token into lowercase 
    for i in range(len(tokens)):
        
        tokens[i] = tokens[i].lower()
        
        tokens[i] = re.sub(r'\W+', '', tokens[i])

    for words in tokens: 
        desc_dict[key].append(words)

In [69]:
#get wordset

for i,j in enumerate(desc_dict.keys()):
    if i == 0: wordset = np.union1d(desc_dict[j], desc_dict[j])
    else: wordset = np.union1d(wordset, desc_dict[j])

In [70]:
len(wordset), wordset

(10989, array(['', '0', '02', ..., '𝕨𝕒𝕤', '𝗶𝗹𝘆', '𝚊𝚜𝚖𝚛ツ'], dtype='<U419'))

In [71]:
#assign word frequency (create function)

def calculateBOW(wordset, desc_list):
    tf_diz = dict.fromkeys(wordset,0)
    for word in desc_list:
        tf_diz[word]=desc_list.count(word)
    return tf_diz

In [72]:
#assign word frequency

data = []
for i,j in enumerate(desc_dict.keys()):
    data.append(calculateBOW(wordset, desc_dict[j]))
    
df_bow = pd.DataFrame(data)

In [73]:
df_bow = df_bow.drop([''], axis=1)

In [74]:
df_bow

0  02  021__012  027  0304  05mar22  06  063  09042022часть  0906  ...  \
0     0   0         0    0     0        0   0    0              0     0  ...   
1     0   0         0    0     0        0   0    0              0     0  ...   
2     0   0         0    0     0        0   0    0              0     0  ...   
3     0   0         0    0     0        0   0    0              0     0  ...   
4     0   0         0    0     0        0   0    0              0     0  ...   
...  ..  ..       ...  ...   ...      ...  ..  ...            ...   ...  ...   
5112  0   0         0    0     0        0   0    0              0     0  ...   
5113  0   0         0    0     0        0   0    0              0     0  ...   
5114  0   0         0    0     0        0   0    0              0     0  ...   
5115  0   0         0    0     0        0   0    0              0     0  ...   
5116  0   0         0    0     0        0   0    0              0     0  ...   

      𝕕𝕒𝕟𝕔𝕖  𝕕𝕠𝕟𝕥  𝕗𝕪𝕡  𝕙𝕚  𝕠𝕥  𝕠𝕦𝕥  𝕥𝕙𝕚𝕟𝕜  𝕨𝕒𝕤  𝗶𝗹𝘆  𝚊𝚜𝚖𝚛ツ  
0         0     0    0   0   0    0      0    0    0      0  
1         0     0    0   0   0    0      0    0    0      0  
2         0     0    0   0   0    0      0    0    0      0  
3         0     0    0   0   0    0      0    0    0      0  
4         0     0    0   0   0    0      0    0    0      0  
...     ...   ...  ...  ..  ..  ...    ...  ...  ...    ...  
5112      0     0    0   0   0    0      0    0    0      0  
5113      0     0    0   0   0    0      0    0    0      0  
5114      0     0    0   0   0    0      0    0    0      0  
5115      0     0    0   0   0    0      0    0    0      0  
5116      0     0    0   0   0    0      0    0    0      0  

[5117 rows x 10988 columns]

In [75]:
df_bow.describe()

0                     02               021__012  \
count 5,117.0000000000000000 5,117.0000000000000000 5,117.0000000000000000   
mean      0.0005862810240375     0.0001954270080125     0.0001954270080125   
std       0.0242085073673259     0.0139795210222849     0.0139795210222849   
min       0.0000000000000000     0.0000000000000000     0.0000000000000000   
25%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
50%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
75%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
max       1.0000000000000000     1.0000000000000000     1.0000000000000000   

                         027                   0304                05mar22  \
count 5,117.0000000000000000 5,117.0000000000000000 5,117.0000000000000000   
mean      0.0001954270080125     0.0001954270080125     0.0001954270080125   
std       0.0139795210222849     0.0139795210222849     0.0139795210222849   
min       0.0000000000000000     0.0000000000000000     0.0000000000000000   
25%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
50%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
75%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
max       1.0000000000000000     1.0000000000000000     1.0000000000000000   

                          06                    063          09042022часть  \
count 5,117.0000000000000000 5,117.0000000000000000 5,117.0000000000000000   
mean      0.0001954270080125     0.0001954270080125     0.0001954270080125   
std       0.0139795210222849     0.0139795210222849     0.0139795210222849   
min       0.0000000000000000     0.0000000000000000     0.0000000000000000   
25%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
50%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
75%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
max       1.0000000000000000     1.0000000000000000     1.0000000000000000   

                        0906  ...                  𝕕𝕒𝕟𝕔𝕖  \
count 5,117.0000000000000000  ... 5,117.0000000000000000   
mean      0.0001954270080125  ...     0.0001954270080125   
std       0.0139795210222849  ...     0.0139795210222849   
min       0.0000000000000000  ...     0.0000000000000000   
25%       0.0000000000000000  ...     0.0000000000000000   
50%       0.0000000000000000  ...     0.0000000000000000   
75%       0.0000000000000000  ...     0.0000000000000000   
max       1.0000000000000000  ...     1.0000000000000000   

                        𝕕𝕠𝕟𝕥                    𝕗𝕪𝕡                     𝕙𝕚  \
count 5,117.0000000000000000 5,117.0000000000000000 5,117.0000000000000000   
mean      0.0001954270080125     0.0001954270080125     0.0001954270080125   
std       0.0139795210222849     0.0139795210222849     0.0139795210222849   
min       0.0000000000000000     0.0000000000000000     0.0000000000000000   
25%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
50%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
75%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
max       1.0000000000000000     1.0000000000000000     1.0000000000000000   

                          𝕠𝕥                    𝕠𝕦𝕥                  𝕥𝕙𝕚𝕟𝕜  \
count 5,117.0000000000000000 5,117.0000000000000000 5,117.0000000000000000   
mean      0.0001954270080125     0.0001954270080125     0.0001954270080125   
std       0.0139795210222849     0.0139795210222849     0.0139795210222849   
min       0.0000000000000000     0.0000000000000000     0.0000000000000000   
25%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
50%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
75%       0.0000000000000000     0.0000000000000000     0.0000000000000000   
max       1.0000000000000000     1.000000000000000

In [76]:
word_freq = {}
for i in df_bow.columns:
    word_freq[i] = df_bow.sum()[i]

KeyboardInterrupt: 

In [ ]:
sorted_word_freq = dict(sorted(word_freq.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
{k:v for (k,v) in sorted_word_freq.items() if v > 30}